In [23]:
# pip install v20

In [8]:
API_KEY = "33fd42396255be10b3ce710d543ca8e4-d44bdfab55d6a4fd8ee163ae25ae1744"
ACCOUNT_ID = "101-001-29619431-001"
OANDA_URL = 'https://api-fxpractice.oanda.com/v3'

SECURE_HEADER = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [22]:
import os
import json
import requests
import pandas as pd  # Import pandas for DataFrame operations

# Define environment variables (or set them directly here)
env = {
    "API_ENDPOINT": "https://api-fxpractice.oanda.com",  # Replace with your actual API endpoint
    "ACCESS_TOKEN": "33fd42396255be10b3ce710d543ca8e4-d44bdfab55d6a4fd8ee163ae25ae1744"  # Replace with your actual access token
}

# Define input parameters directly
instrument = "EUR_USD"  # Example instrument
granularity = "H1"      # Example granularity
data_path = "./data"    # Example data path

# Check if the required environment variables are set
base_url = env.get("API_ENDPOINT")
access_token = env.get("ACCESS_TOKEN")

# Function to fetch all candles within the time range
def fetch_all(*, base_url, access_token, instrument, granularity, start_time, end_time):
    include_first = True
    while True:
        page = fetch_page(
            base_url=base_url,
            access_token=access_token,
            instrument=instrument,
            granularity=granularity,
            start_time=start_time,
            include_first=include_first
        )

        # Debug: Print the first candle to understand its structure
        if page["candles"]:
            print("Sample candle structure:", page["candles"][0])

        for candle in page["candles"]:
            if candle["time"] < end_time:
                yield candle
            else:
                return
        if page["candles"][-1]["time"] < end_time:
            start_time = page["candles"][-1]["time"]
            include_first = False

# Function to fetch a single page of candles
def fetch_page(*, base_url, access_token, instrument, granularity, start_time, include_first):
    url = f"{base_url}/v3/instruments/{instrument}/candles"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    params = {
        "price": "BA",  # Use 'BA' to get both bid and ask prices
        "granularity": granularity,
        "from": start_time,
        "count": 5000,
        "includeFirst": include_first
    }
    response = requests.get(url, headers=headers, params=params)
    assert response.status_code == 200, f"{response.status_code}, {response.text}"
    return response.json()

if not base_url or not access_token:
    print("API_ENDPOINT and ACCESS_TOKEN environment variables are required")
else:
    # Proceed with data collection
    print(f"Collecting {instrument} OHLCV data...")
    start_time = "2024-01-01T00:00:00.00Z"
    end_time = "2024-08-02T00:00:00.00Z"

    # Ensure the data directory exists
    os.makedirs(data_path, exist_ok=True)

    # Create a list to store the OHLCV data
    ohlcv_data = []

    # Fetch data and collect OHLCV information
    for candle in fetch_all(
        base_url=base_url,
        access_token=access_token,
        instrument=instrument,
        granularity=granularity,
        start_time=start_time,
        end_time=end_time
    ):
        # Calculate mid prices as the average of bid and ask
        ohlcv = {
            "time": candle["time"],
            "open": (float(candle["bid"]["o"]) + float(candle["ask"]["o"])) / 2,
            "high": (float(candle["bid"]["h"]) + float(candle["ask"]["h"])) / 2,
            "low": (float(candle["bid"]["l"]) + float(candle["ask"]["l"])) / 2,
            "close": (float(candle["bid"]["c"]) + float(candle["ask"]["c"])) / 2,
            "volume": candle["volume"]
        }
        ohlcv_data.append(ohlcv)  # Append the OHLCV data to the list

    # Create a DataFrame from the OHLCV data
    df = pd.DataFrame(ohlcv_data)

    # Print the DataFrame for verification
    print(df.head())

    # Save the DataFrame to a CSV file
    csv_filename = os.path.join(data_path, f"{instrument}_{granularity}_ohlcv.csv")
    df.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

    # Alternatively, save the DataFrame to an Excel file
    excel_filename = os.path.join(data_path, f"{instrument}_{granularity}_ohlcv.xlsx")
    df.to_excel(excel_filename, index=False)
    print(f"Data saved to {excel_filename}")


Sample candle structure: {'complete': True, 'volume': 188, 'time': '2024-01-01T22:00:00.000000000Z', 'bid': {'o': '1.10416', 'h': '1.10444', 'l': '1.10407', 'c': '1.10438'}, 'ask': {'o': '1.10465', 'h': '1.10477', 'l': '1.10441', 'c': '1.10462'}}
                             time      open      high       low     close  \
0  2024-01-01T22:00:00.000000000Z  1.104405  1.104605  1.104240  1.104500   
1  2024-01-01T23:00:00.000000000Z  1.104455  1.104475  1.103565  1.103675   
2  2024-01-02T00:00:00.000000000Z  1.103670  1.103855  1.103425  1.103835   
3  2024-01-02T01:00:00.000000000Z  1.103800  1.104335  1.103425  1.103500   
4  2024-01-02T02:00:00.000000000Z  1.103510  1.103590  1.101840  1.101895   

   volume  
0     188  
1    1018  
2    1328  
3    2435  
4    1968  
Data saved to ./data\EUR_USD_H1_ohlcv.csv
Data saved to ./data\EUR_USD_H1_ohlcv.xlsx


In [27]:
import os
import json
import requests
import pandas as pd
import numpy as np
import time

# Define environment variables (or set them directly here)
env = {
    "API_ENDPOINT": "https://api-fxpractice.oanda.com",  # Replace with your actual API endpoint
    "ACCESS_TOKEN": "33fd42396255be10b3ce710d543ca8e4-d44bdfab55d6a4fd8ee163ae25ae1744"  # Replace with your actual access token
}

# Define input parameters directly
instrument = "EUR_USD"  # Example instrument
granularity = "M1"      # Example granularity for real-time (1-minute intervals)
data_path = "./data"    # Example data path

# Trading parameters
rsi_period = 14
rsi_oversold = 30
rsi_overbought = 70

# OANDA API endpoints
base_url = env.get("API_ENDPOINT")
access_token = env.get("ACCESS_TOKEN")

# Function to fetch the latest candles
def fetch_latest_candles(base_url, access_token, instrument, granularity, count=20):
    url = f"{base_url}/v3/instruments/{instrument}/candles"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    params = {
        "price": "BA",  # Use 'BA' to get both bid and ask prices
        "granularity": granularity,
        "count": count
    }
    response = requests.get(url, headers=headers, params=params)
    assert response.status_code == 200, f"{response.status_code}, {response.text}"
    return response.json()

# Function to calculate RSI
def calculate_rsi(data, period=14):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to execute trade orders
def execute_order(base_url, access_token, instrument, units):
    url = f"{base_url}/v3/accounts/YOUR_ACCOUNT_ID/orders"  # Replace YOUR_ACCOUNT_ID with your actual account ID
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    order = {
        "order": {
            "units": str(units),  # Positive for buy, negative for sell
            "instrument": instrument,
            "timeInForce": "FOK",
            "type": "MARKET",
            "positionFill": "DEFAULT"
        }
    }
    response = requests.post(url, headers=headers, json=order)
    assert response.status_code == 201, f"Order failed: {response.status_code}, {response.text}"
    return response.json()

# Main trading loop
if base_url and access_token:
    print(f"Starting real-time trading for {instrument}...")
    current_position = 0  # Track current position: 0 = no position, 1 = long, -1 = short

    try:
        while True:
            # Fetch latest candle data
            candle_data = fetch_latest_candles(base_url, access_token, instrument, granularity)
            candles = candle_data['candles']
            
            # Extract and calculate OHLCV
            ohlcv_data = []
            for candle in candles:
                ohlcv = {
                    "time": candle["time"],
                    "open": (float(candle["bid"]["o"]) + float(candle["ask"]["o"])) / 2,
                    "high": (float(candle["bid"]["h"]) + float(candle["ask"]["h"])) / 2,
                    "low": (float(candle["bid"]["l"]) + float(candle["ask"]["l"])) / 2,
                    "close": (float(candle["bid"]["c"]) + float(candle["ask"]["c"])) / 2,
                    "volume": candle["volume"]
                }
                ohlcv_data.append(ohlcv)
            
            # Create a DataFrame and calculate RSI
            df = pd.DataFrame(ohlcv_data)
            df['rsi'] = calculate_rsi(df, period=rsi_period)
            
            # Get the latest RSI value
            latest_rsi = df['rsi'].iloc[-1]
            latest_close = df['close'].iloc[-1]
            print(f"Latest RSI: {latest_rsi}, Latest Close: {latest_close}")

            # Trading logic
            if latest_rsi < rsi_oversold and current_position != 1:
                # Buy signal
                print(f"RSI {latest_rsi} is below {rsi_oversold}, placing a buy order...")
                execute_order(base_url, access_token, instrument, units=1000)  # Buy 1000 units
                current_position = 1
            elif latest_rsi > rsi_overbought and current_position != -1:
                # Sell signal
                print(f"RSI {latest_rsi} is above {rsi_overbought}, placing a sell order...")
                execute_order(base_url, access_token, instrument, units=-1000)  # Sell 1000 units
                current_position = -1

            # Wait for the next interval
            time.sleep(60)  # Sleep for 60 seconds (or the chosen granularity time interval)

    except KeyboardInterrupt:
        print("Trading stopped manually.")
    except Exception as e:
        print(f"An error occurred: {e}")

else:
    print("API_ENDPOINT and ACCESS_TOKEN environment variables are required")

Starting real-time trading for EUR_USD...
Latest RSI: 52.2388059701453, Latest Close: 1.09111
Latest RSI: 52.2388059701453, Latest Close: 1.09111
Latest RSI: 52.2388059701453, Latest Close: 1.09111
Latest RSI: 52.2388059701453, Latest Close: 1.09111
Latest RSI: 52.2388059701453, Latest Close: 1.09111
Trading stopped manually.
